<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/train_llama2_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## llama2.c

你是否曾想过在纯C中推理一个[Llama 2](https://ai.meta.com/llama/)模型？不是吗？现在你可以了！

使用PyTorch训练Llama 2 LLM架构，然后通过一个简单的700行C文件([run.c](run.c))进行推理。你可能认为你需要许多十亿参数的LLMs才能做任何有用的事情，但事实上，如果你将领域限制得足够窄（参考：[TinyStories](https://huggingface.co/datasets/roneneldan/TinyStories)论文），非常小的LLMs可能表现出人意料的强大性能。这个仓库是一个"Llama 2 LLM"的完整训练+推理解决方案，重点放在极简和简单上。

由于架构相同，你也可以加载和推理Meta的Llama 2模型。然而，当前的代码只推理fp32模型，因此你可能无法有效地加载大于7B的模型。模型量化的工作正在进行中。

请注意，这个仓库最近作为一个有趣的周末项目开始：我拿出了之前的[nanoGPT](https://github.com/karpathy/nanoGPT)，调整它以实现Llama-2架构而不是GPT-2，其中的核心是在[run.c](run.c)中编写C推理引擎。因此，这个项目还很年轻，正在快速发展。向令人印象深刻的[llama.cpp](https://github.com/ggerganov/llama.cpp)致敬，它激发了这个项目。与llama.cpp相比，我想要的是一些非常简单、极简和有教育意义的东西，所以我选择硬编码Llama 2架构，只使用纯C编写一个推理文件，没有任何依赖。

## 感受魔法

[![在Colab中打开](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/karpathy/llama2.c/blob/master/run.ipynb)

首先，转到保存项目的文件夹，并将此存储库克隆到该文件夹：

```bash
git clone https://github.com/karpathy/llama2.c.git
```

然后，打开存储库文件夹：

```bash
cd llama2.c
```

现在，让我们在C中运行一个小型的Llama 2模型。你需要一个模型检查点。下载我在[TinyStories](https://huggingface.co/datasets/roneneldan/TinyStories)数据集上训练的这个1500万参数模型（约60MB下载）：

```bash
wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin
```

编译并运行C代码：

```bash
make run
./run stories15M.bin
```

你将看到文本流的一个样本。在我的M1 MacBook Air上，运行速度约为每秒110个标记。查看[performance](#performance)或Makefile以了解可以显著加速此过程的编译标志。我们还可以尝试一个稍大的4200万参数模型：

```bash
wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin
./run stories42M.bin
```
这仍然以交互速率运行，并生成更连贯和多样化的故事样本：

> Once upon a time, there was a little girl named Lily. She loved playing with her toys on top of her bed. One day, she decided to have a tea party with her stuffed animals. She poured some tea into a tiny teapot and put it on top of the teapot. Suddenly, her little brother Max came into the room and wanted to join the tea party too. Lily didn't want to share her tea and she told Max to go away. Max started to cry and Lily felt bad. She decided to yield her tea party to Max and they both shared the teapot. But then, something unexpected happened. The teapot started to shake and wiggle. Lily and Max were scared and didn't know what to do. Suddenly, the teapot started to fly towards the ceiling and landed on the top of the bed. Lily and Max were amazed and they hugged each other. They realized that sharing was much more fun than being selfish. From that day on, they always shared their tea parties and toys.

你还可以通过前缀或一些额外的命令行参数提示模型，例如以0.8的温度进行256步的采样并带有一个提示：

```bash
./run stories42M.bin -t 0.8 -n 256 -i "有一天，莉莉遇到了一只肖戈斯"
```

> One day, Lily met a Shoggoth. He was very shy, but was also very generous. Lily said “Hello Shoggy! Can I be your friend?” Shoggy was happy to have a friend and said “Yes, let’s explore the universe together!” So they set off on a journey to explore the universe. As they travelled, Shoggy was happy to explain to Lily about all the wonderful things in the universe. At the end of the day, Lily and Shoggy had gathered lots of wonderful things from the universe, and they both felt very proud. They promised to explore the universe as one big pair and to never stop being generous to each other.

还有一个更好的110M参数模型可用，详见[models](#models)。

关于采样的快速提示，推荐的最佳结果是使用`-t 1.0 -p 0.9`进行采样，即温度1.0（默认值），同时进行0.9的top-p采样（默认值）。直观地说，top-p确保不会对概率微小的标记进行采样，因此我们在采样时不会“不幸”，之后也不太可能“偏离正轨”。总的来说，要控制样本的多样性，请使用温度（即在0和1之间变化的`-t`并关闭top-p，使用`-p 0`）或top-p值（即在0和1之间变化的`-p`并保持`-t 1`），但不要同时使用两者。关于LLM采样策略的详细解释包括[这个](https://peterchng.com/blog/2023/05/02/token-selection-strategies-top-k-top-p-and-temperature/)，[这个](https://docs.cohere.com/docs/controlling-generation-with-top-k-top-p)或[这个](https://huggingface.co/blog/how-to-generate)。

## Meta的Llama 2模型

由于神经网络架构相同，我们也可以推理Meta发布的Llama 2模型。不幸的是，由于许可证问题（我不能直接上传检查点，我想），这里存在一些摩擦。所以第1步，按照[Meta的说明](https://github.com/facebookresearch/llama)获取Llama 2检查点。一旦我们获得了这些检查点，我们必须将它们转换为llama2.c格式。
为此，我们需要安装python依赖项（`pip install -r requirements.txt`），然后使用`export.py`文件，例如对于7B模型：

```bash
python export.py llama2_7b.bin --meta-llama path/to/llama/model/7B
```

导出将花费大约10分钟左右，并生成一个26GB的文件（7B模型的float32权重），称为当前目录中的`llama2_7b.bin`。有[报道](https://github.com/karpathy/llama2.c/pull/85)称，尽管有努力，但目前不要尝试运行7B以上的任何模型，原因有两个：首先，13B+目前由于指针算术中的整数溢出而无法正常工作，这还有待修复，其次，即使修复了，此仓库目前正在进行float32推理，因此速度相当慢，几乎无法使用。导出完成后，我们可以运行它：

```bash
./run llama2_7b.bin
```

在我的Linux云端CPU上，使用[OpenMP](#OpenMP)编译并行96线程，运行速度约为每秒4个标记。（在我的MacBook Air M1上，如果只是使用`make runfast`构建，目前每个标记大约需要30秒。）示例输出：

> The purpose of this document is to highlight the state-of-the-art of CoO generation technologies, both recent developments and those in commercial use. The focus is on the technologies with the highest merit to become the dominating processes of the future and therefore to be technologies of interest to S&amp;T ... R&amp;D. As such, CoO generation technologies developed in Russia, Japan and Europe are described in some depth. The document starts with an introduction to cobalt oxides as complex products and a short view on cobalt as an essential material. The document continues with the discussion of the available CoO generation processes with respect to energy and capital consumption as well as to environmental damage.

基础模型... ¯\\_(ツ)_/¯。由于我们可以推理基础模型，因此很容易推理聊天模型，并与其进行对话。如果我们能找到更有效地运行7B的方法，我们可以开始在仓库中添加LoRA到我们的训练脚本，并在其中进行精调！

你也可以与Llama Chat模型聊天。按照上面的步骤导出聊天模型：

```bash
python export.py llama2_7b_chat.bin --meta-llama /path/to/7B-chat
```

然后通过使用`-m`标志指定聊天模式进行聊天，例如：

```bash
./run llama2_7b_chat.bin -m chat
```

你也可以尝试Meta的Code Llama模型，即使对它们的支持不完整。特别是一些超参数发生了变化（例如RoPE层中的常数），因此推理目前不是完全正确的，而且有点错误。正在研究修复方法。确保为普通和指导变体构建标记器，并在进行推理时传递它。

```bash
python export.py codellama2_7b.bin --meta-llama /path/to/CodeLlama-7b
python tokenizer.py --tokenizer-model=/path/to/CodeLlama-7b/tokenizer.model
./run codellama2_7b.bin -z /path/to/CodeLlama-7b/tokenizer.bin
```

与Code Llama Instruct聊天：

```bash
python export.py codellama2_7b_instruct.bin --meta-llama /path/to/CodeLlama-7b-Instruct
python tokenizer.py --tokenizer-model=/path/to/CodeLlama-7b-Instruct/tokenizer.model
./run codellama2_7b_instruct.bin -m chat -z /path/to/CodeLlama-7b-Instruct/tokenizer.bin
```

## int8量化

上面的（默认）脚本[run.c](run.c)使用了float32前向传播，其中整个前向传播的计算都保持在fp32中。就参考代码而言，这非常容易理解，但它具有以下缺点：模型检查点文件非常大（每个单独权重占用4个字节），并且前向传播相对较慢。实际上，非常常见的推理优化方法是将模型参数量化为较低的精度，以获取较小的检查点大小和更快的前向传播（因为大多数推理使用整数运算），换取一点点正确性。经验上，LLMs可以容忍低至4位（甚至更低）的精度，但在这里我们使用int8，因为这是一个“安全”的设置，既可以带来好处，又不会牺牲太多模型准确性。只有参与矩阵乘法的权重被量化。所有其他参数（例如RMSNorm中的尺度和偏差）都保持为float32，因为这些层非常敏感。现在，如果你只是想要减小检查点的大小，你可以量化权重，保存检查点，然后在run.c中对它们进行反量化，并像正常一样进行float32推理，然后结束。这完全没问题。但在这里，我们更进一步（正如标准做法一样），并在前向传播中另外量化激活。这要求我们在运行时在float32和int8之间动态量化和反量化，这增加了开销。但好处是现在大多数计算（尤其是矩阵乘法！）都使用纯整数运算，其中权重和激活都以int8形式输入。这基本上是加速的本质所在。我们使用的版本是“Q8_0”量化（llama.cpp术语），其中0表示权重量化围绕0对称，量化到范围[-127，127]。

量化的前向传播实现在[runq.c](runq.c)中。要使用它，我们必须以量化格式导出模型。例如，Llama 2 7B的float32版本被导出为：

```
python export.py llama2_7b.bin --meta-llama path/to/llama/model/7B
```

这会创建一个26GB的文件，因为每个7B参数都是4个字节（fp32）。要导出为量化格式，我们改用版本2导出：

```
python export.py llama2_7b_q80.bin --version 2 --meta-llama path/to/llama/model/7B
```

这运行了几分钟，但现在只创建了一个6.7GB的文件。对于导出非meta检查点，您将使用--meta-llama arg的--checkpoint arg（稍后在下面的文档中有更多信息）。现在让我们进行推理。在这里我喜欢使用OMP因为这些是大模型，所以例如在我的Linux桌面上：

```
make runomp
OMP_NUM_THREADS=64 ./run llama2_7b.bin -n 40
OMP_NUM_THREADS=64 ./runq llama2_7b_q80.bin -n 40
```

这运行了40步，只是为了获得一个时间。对我来说，float32版本的运行速度为4.6标记/秒，而int8版本的运行速度为14标记/秒。因此，我们实现了3倍的加速，同时将检查点大小减小了4倍。然而，前向传播被量化为int8，因此在很小程度上降低了质量。

## huggingface模型

我们可以加载使用Llama 2架构的任何huggingface模型。请查看脚本[export.py](export.py)和`--hf`标志以导出模型的.bin文件。

## 模型

出于对从头开始模型的示例的考虑，我在TinyStories上训练了一系列小模型。所有这些在我的训练设置上（4X A100 40GB GPU）几小时内完成。110M模型花费了大约24小时。我将它们托管在huggingface hub [tinyllamas](https://huggingface.co/karpathy/tinyllamas)上，既在原始的PyTorch .pt格式中，也在llama2.c格式的.bin文件中：

| model | dim | n_layers | n_heads | n_kv_heads | max context length | parameters | val loss | download
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 260K | 64 | 5 | 8 | 4 | 512 | 260K | 1.297 | [stories260K](https://huggingface.co/karpathy/tinyllamas/tree/main/stories260K)
| OG | 288 | 6 | 6 | 6 | 256 | 15M | 1.072 | [stories15M.bin](https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin) |
| 42M| 512 | 8 | 8 | 8 | 1024 | 42M | 0.847 | [stories42M.bin](https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin) |
| 110M| 768 | 12 | 12 | 12 | 1024 | 110M | 0.760 | [stories110M.bin](https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin) |

你会注意到110M模型的大小相当于GPT-1。或者说，这也是GPT-2系列中最小的模型（`GPT-2 small`），只是最大上下文长度仅为1024而不是2048。与GPT-1/2架构的唯一显著变化是，Llama使用RoPE相对位置嵌入，而不是绝对/学习位置嵌入，在MLP中使用了更炫的SwiGLU非线性，RMSNorm代替LayerNorm，所有Linear层上都使用了bias=False，而且是可选的多查询（但这在llama2.c中尚不受支持）。

## 训练

让我们看看如何使用此存储库中的代码从头开始训练一个baby Llama 2模型。首先，让我们下载并对一些源数据集进行预分词，例如，我喜欢[TinyStories](https://huggingface.co/datasets/roneneldan/TinyStories)，所以这是此存储库中当前唯一可用的示例。但应该很容易添加数据集，查看代码。

```bash
python tinystories.py download
python tinystories.py pretokenize
```

然后训练我们的模型：

```bash
python train.py
```

**简要的训练指南**。查看train.py脚本以获取更多异类启动和超参数覆盖。以下是如何设置参数的简要指南。查看[Chinchilla paper](https://arxiv.org/abs/2203.15556)末尾的表格，了解Transformer参数（dim，n_layers，n_heads）是如何一起增长或缩小的。推断/插值此模式以获得更大或更小的Transformer。设置最大上下文长度，取决于问题：这应该是预测下一个标记所需的最大标记数。例如，Llama 2使用2048。接下来，您希望每次更新的_total_批量大小（脚本打印为“tokens per iteration will be:”）在中等应用中大约为100K标记。对于微小应用程序，它可能更低，对于大型训练（例如GPTs/LLamas），它通常为〜0.5M，甚至更多。您通过首先将batch_size最大化到系统允许的任何值（例如，在最近的一次运行中，我的GPU在16之后就没有内存了），然后您想增加gradient_accumulation_steps，使其足够高以达到总批量大小为〜100K的标记数。最后，您希望调整learning_rate（LR）。您希望此值尽可能高。非常小的网络可以使用较大的LR（例如1e-3甚至更高）。大型网络需要较低的LR。对于大多数中等大小的应用程序，3e-4是一个安全的选择，但对于小网络来说可能太低了，所以尽量增加它！最后，max_iters是训练的长度。尝试不同的设置。我主要只调整这些参数，而将大多数其他参数保持不变。这是我如何训练110M模型的示例，我认为这还远非最佳，但在我看来是明智的：dim 768，n_layers 12，n_heads 12（因此每个头的大小为768 / 12 = 64通道），seq len为1024，batch size为16（这是我A100 40GB GPU最大的值），gradient_accumulation_steps = 8需要将总tokens批量大小增加到16批次大小 * 1024个标记的序列 * 8 grad_accum = 131,072每次更新的标记。很好。学习率4e-4（可能稍微偏低）。max_iters 200K（可能稍微偏高）。Dropout 0.1，因为在中等大小时通常有点帮助。就是这样。我在我的云机器上使用了分布式数据并行（DDP）在4个GPU上运行，培训花了大约一天左右。

如果你只是想要进行一个简单的演示而不想进行模型训练，完全可以跳过模型训练部分，只需下载其中一个预训练模型（参见[models](#models)部分），例如：

```bash
wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin
```

一旦我们有了model.bin文件，我们可以在C中进行推理。首先编译C代码：

```bash
make run
```

现在，你可以简单地运行它：

```bash
./run stories15M.bin
```

观察标记流动，很有趣！我们还可以运行PyTorch推理脚本进行比较。再次从huggingface hub下载其中一个模型，并将`sample.py`脚本指向它：

```bash
wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.pt -P out15M
python sample.py --checkpoint=out15M/stories15M.pt
```

这会得到相同的结果。

## 自定义 tokenizers

在上述所有内容中，我们假设了带有32,000个标记的自定义Lllama 2分词器。然而，在许多精品LLM中，使用这么大的词汇可能有些过分。如果你有一个小的应用程序，你可能更好地训练自己的分词器。这可以使一切更好 - 用较小的词汇，你的模型参数更少（因为标记嵌入表要小得多），推理更快（因为要预测的标记更少），每个示例的平均序列长度也可能更小（因为对数据的压缩更有效）。所以让我们看看如何训练一个自定义分词器。

默认情况下，要对tinystories数据集进行预分词，我们必须按顺序运行以下命令：

```
python tinystories.py download
python tinystories.py pretokenize
```

这里的`pretokenize`阶段加载了Llama 2分词器（词汇大小为32,000），并使用它将下载的文本转换为整数，并将其保存到文件中。我们现在将其更改如下，以训练一个示例4096标记的分词器：

```
python tinystories.py download
python tinystories.py train_vocab --vocab_size=4096
python tinystories.py pretokenize --vocab_size=4096
```

`train_vocab`阶段将调用`sentencepiece`库来训练分词器，并将其存储在新文件`data/tok4096.model`中。我尽量复制了（我认为）Meta用于训练他们的词汇的设置。这使用了字节对编码算法，该算法从文本数据的原始utf8字节序列开始，然后迭代地合并最常见的连续标记对以形成词汇表。检查`tinystories.py`文件 - 自定义分词器存储在一个特殊的目录结构中，由词汇大小索引。

有趣的一点是，专门在tinystories上训练的4096个标记的词汇大小会创建大约与默认的32,000标记的Llama 2分词器相同的每个示例的序列长度！这意味着我们的定制分词器更适应我们特定的文本，可以非常有效地压缩它。因此，我们训练的模型更小更快。

现在我们使用自定义分词器对数据集进行了预分词化，我们可以训练模型。训练脚本`train.py`不关心确切的标记，它只关心词汇大小，以便可以正确初始化模型。因此，在训练模型时，请确保传入

```
python train.py --vocab_source=custom --vocab_size=4096
```

（默认值分别为`llama2`和`32000`，表示默认的Llama 2分词器）。这样就训练好了模型。最后，我们准备使用我们的`run.c`脚本进行推理。为此，我们需要两件事。首先，我们必须以`.bin`格式导出我们的分词器，使用以下命令：

```
python tokenizer.py --tokenizer-model=data/tok4096.model
```

这将将分词器写入`data/tok4096.bin`。现在我们可以运行推理，使用`-z`标志指向这个分词器：

```
./run out/model.bin -z data/tok4096.bin
```

这应该打印出样本。如果省略`-z`标志，它将使用默认的Llama 2分词器，这将生成一系列好的整数，但是它们将使用不同的词汇表被翻译成文本，所以看起来像胡言乱语。

## performance

有很多方法可以根据你的系统可能加速这段代码。查看[Makefile](Makefile)，其中包含许多注释。`make run`命令当前默认使用`-O3`优化，即：

```bash
gcc -O3 -o run run.c -lm
```

`-O3`包含在编译时间和内存使用方面昂贵的优化。包括矢量化、循环展开和分支预测。

为了获得更好的性能，尝试使用`make runfast`进行编译。这将打开`-Ofast`标志，该标志包括可能违反C/IEEE规范的额外优化，除了`-O3`。请参阅[GNU GCC文档](https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html)以获取更多信息。

尝试使用`-march=native`将程序编译为使用正在编译的机器的体系结构，而不是更通用的CPU。这可能启用附加优化和硬件特定的调整，例如改进的矢量指令/宽度。

到目前为止，在我的MacBook Air（M1）上，最快的吞吐量是使用`make runfast`。

你还可以尝试用`clang`替换`gcc`进行实验。

如果使用gcc进行编译，请尝试使用`-funroll-all-loops`进行实验，参见PR [#183](https://github.com/karpathy/llama2.c/pull/183)。

**OpenMP**。通过使用OpenMP编译，还可以实现显著的性能改进，该编译“激活”了matmul和attention内的`#pragma omp parallel for`，允许在多个处理器上拆分循环中的工作。
首先，你需要安装OpenMP库和clang编译器（例如，在ubuntu上运行`apt install clang libomp-dev`）。然后，你可以使用`make runomp`进行编译，该命令执行：

```bash
clang -Ofast -fopenmp -march=native run.c  -lm  -o run
```

在运行推理时，请确保使用OpenMP标志设置线程数，例如：

```bash
OMP_NUM_THREADS=4 ./run out/model.bin
```

根据你的系统资源，你可能希望调整这些超参数并使用更多线程。但并不总是越多越好，通常是U形状的。特别是，如果你的CPU支持SMT（多线程），请尝试将线程数设置为物理核心数而不是逻辑核心数。由于缓存抖动和通信开销，性能差异可能很大。PyTorch文档[CPU特定优化](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html#cpu-specific-optimizations)中也有一些在这里适用的好信息。

## platforms

在**Windows**上，使用Visual Studio命令提示符中的`build_msvc.bat`使用msvc进行构建，或者你可以使用`make win64`从linux或windows使用mingw编译器工具链构建windows目标。MSVC构建将自动使用适合你的CPU的openmp和最大线程数，除非你设置了`OMP_NUM_THREADS`环境变量。

在**Centos 7**、**Amazon Linux 2018**上，使用`rungnu` Makefile目标：`make rungnu`或`make runompgnu`使用openmp。

在**Mac**上，使用brew安装clang进行openmp构建。安装clang为`brew install llvm`，并使用安装的clang二进制文件进行带有openmp的编译：`make runomp CC=/opt/homebrew/opt/llvm/bin/clang`

## tests

你可以使用pytest简单地运行测试：

```bash
$ pip install pytest
$ pytest
```

这将当前调用`test_all.py`中的两个测试，分别在C和Python中推进模型200步，并将输出与已知的期望输出进行比较。当前测试仅需几秒钟，但必须在临时`test`目录中下载和缓存stories260K模型（仅约2MB下载）。

还有一些C语言的测试，位于[test.c](test.c)文件中。你可以使用`make testcc`运行这些测试，或者查看更多打印的内容：

```
make testcc VERBOSITY=1
```

请求帮助：帮助添加更多测试。

## ack

我在一台由优秀的[Lambda labs](https://lambdalabs.com/service/gpu-cloud)慷慨提供的4块A100 40GB显卡的机器上训练了llama2.c的故事生成模型，感谢他们。

## discord

我想到可以重复使用我现有的Discord频道（我在其中进行[从零到英雄的YouTube系列](https://karpathy.ai/zero-to-hero.html)），请查看[discord](https://discord.gg/3zy8kqD9Cp)上的#llama2c频道，用于任何快速问题、相关讨论等。

## contributing
关于这个仓库及可能被接受的PR的一些建议。这个仓库的目标是什么？基本上，我认为很多人会对在各种各样的应用中训练或微调自定义微型LLM（大约100M - 1B参数，但我们说最多可以达到10B参数）感兴趣，并在边缘附近的环境中部署它们（例如MCUs、手机、Web浏览器、笔记本电脑等）。我希望这个仓库是支持这种工作流程（包括训练和推断）的最简单、最小、最易于操作的仓库。特别是，这个仓库不是一个复杂的框架，控制着一个嵌套的数百文件的目录结构中难以理解的代码，有着上千个旋钮。相反，我预计大多数应用程序都希望创建这个仓库的一个分支，并将其修改为满足其特定需求和部署平台的形式。

最关心部署效率的人应该看看[llama.cpp](https://github.com/ggerganov/llama.cpp)。这个仓库仍然关注效率，但不以简洁、可读性或可移植性为代价。基本上，我期望很多人来到这个仓库，因为训练代码是2个可读的.py文件，推断代码是500行的C代码。因此，我希望这继续是一种最简单的“参考实现”，可以轻松地在一个单独的分支中进行修改，以适应人们激动的下游应用。它不应该是功能齐全的。它不应该有100个不同的选项或设置。它不应该是最高效的。以下是一些示例：

- 有人重新排序了两个循环，以改善数据局部性，获得了小幅的效率提升 => 立即合并。
- 有人添加了一行"pragma omp parallel for"，允许您使用OpenMP编译，并显著加速代码，或者如果不以这种方式编译则作为注释 => 立即合并。
- 修复错误和润色等 => 非常愿意合并

不太适合的一些PR的示例：

- 在代码的各个地方添加了多个#ifdefs。如果它们是局部的/很少的，可能可以接受。
- 添加了很多特定于某个特定平台的代码（例如MCUs，或某个特殊版本的linux或处理器）。这些可能更适合该项目的分支，我非常乐意在下面的部分中维护这些分支的列表。
- 在run.c中添加了数百行代码，这些代码仅在特定情景或平台上激活。

如果您的候选PR包含这些元素，这并不意味着它们不会被合并，只是意味着它们会进入灰色地带。简而言之，我渴望合并任何基本上小的、局部的、广泛适用的、清晰的改变，以提高仓库的效率和可移植性，同时保持其易操作性和可读性。感谢所有寻求帮助我改进项目的PR，谢谢！<3。

In [ ]:
!git clone https://github.com/karpathy/llama2.c

Cloning into 'llama2.c'...
remote: Enumerating objects: 1434, done.
remote: Total 1434 (delta 0), reused 0 (delta 0), pack-reused 1434
Receiving objects: 100% (1434/1434), 1.17 MiB | 27.19 MiB/s, done.
Resolving deltas: 100% (871/871), done.


In [ ]:
%cd llama2.c

/content/llama2.c


In [ ]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#下载tinystories 数据集
!python tinystories.py download

data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [02:00<00:00, 13.4MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [ ]:
# 对tinystories数据集进行预标记
!python tinystories.py pretokenize

In [ ]:
!ls -hlg data/TinyStories_all_data

total 8.5G
-rw-r--r-- 1 root       40M Dec 22 02:03 data00.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data00.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data01.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data01.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data02.bin
-rw-r--r-- 1 300000513 135M May 17  2023 data02.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data03.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data03.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data04.bin
-rw-r--r-- 1 300000513 135M May 17  2023 data04.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data05.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data05.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data06.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data06.json
-rw-r--r-- 1 root       40M Dec 22 02:03 data07.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data07.json
-rw-r--r-- 1 root       40M Dec 22 02:04 data08.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data08.json
-rw-r--r-- 1 root       40M Dec 22 02:04 dat

In [ ]:
# 训练模型

#!python -m train.py --compile=False --eval_iters=10 --batch_size=80 --dtype=float16
# On T4-16G just example, eval_iters=1 1 eval_iter need 16h or more
!python -m train.py -eval_iters=1 --batch_size=90 --dtype=float16

流式输出内容被截断，只能显示最后 5000 行内容。
16173 | loss 1.2484 | lr 4.715766e-04 | 1374.51ms | mfu 2.07%
16174 | loss 1.1746 | lr 4.715729e-04 | 1365.96ms | mfu 2.07%
16175 | loss 1.2539 | lr 4.715693e-04 | 1375.74ms | mfu 2.07%
16176 | loss 1.1725 | lr 4.715656e-04 | 1372.87ms | mfu 2.07%
16177 | loss 1.2576 | lr 4.715619e-04 | 1366.48ms | mfu 2.08%
16178 | loss 1.2322 | lr 4.715582e-04 | 1371.58ms | mfu 2.08%
16179 | loss 1.2033 | lr 4.715546e-04 | 1367.77ms | mfu 2.08%
16180 | loss 1.2241 | lr 4.715509e-04 | 1373.55ms | mfu 2.08%
16181 | loss 1.1826 | lr 4.715472e-04 | 1374.92ms | mfu 2.08%
16182 | loss 1.2620 | lr 4.715435e-04 | 1375.73ms | mfu 2.08%
16183 | loss 1.2309 | lr 4.715399e-04 | 1368.96ms | mfu 2.08%
16184 | loss 1.2852 | lr 4.715362e-04 | 1373.42ms | mfu 2.08%
16185 | loss 1.2662 | lr 4.715325e-04 | 1369.09ms | mfu 2.08%
16186 | loss 1.2001 | lr 4.715288e-04 | 1368.15ms | mfu 2.08%
16187 | loss 1.1977 | lr 4.715252e-04 | 1372.44ms | mfu 2.08%
16188 | loss 1.2633 | lr 4.715215e-04 | 136

GPU 显存打满
```
nvidia-smi --query-gpu=timestamp,memory.total,memory.free,memory.used,name,utilization.gpu,utilization.memory --format=csv -l 3
timestamp, memory.total [MiB], memory.free [MiB], memory.used [MiB], name, utilization.gpu [%], utilization.memory [%]
2023/12/22 02:48:14.470, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 97 %
2023/12/22 02:48:17.472, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 99 %
2023/12/22 02:48:20.473, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 97 %
2023/12/22 02:48:23.474, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 100 %
2023/12/22 02:48:26.477, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 97 %
2023/12/22 02:48:29.479, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 95 %
2023/12/22 02:48:32.481, 15360 MiB, 331 MiB, 14771 MiB, Tesla T4, 100 %, 97 %
```

# inference 推理

In [ ]:
#直接下载已经训练好的模型
!wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin


--2023-12-23 08:34:42--  https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin
Resolving huggingface.co (huggingface.co)... 13.33.33.55, 13.33.33.102, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/88/4b/884bade32e5ee32eea725c5087af1358179a1bea94a4f6abc3c0470c9610ac38/cd590644d963867a2b6e5a1107f51fad663c41d79c149fbecbbb1f95fa81f49a?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27stories15M.bin%3B+filename%3D%22stories15M.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1703579683&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMzU3OTY4M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84OC80Yi84ODRiYWRlMzJlNWVlMzJlZWE3MjVjNTA4N2FmMTM1ODE3OWExYmVhOTRhNGY2YWJjM2MwNDcwYzk2MTBhYzM4L2NkNTkwNjQ0ZDk2Mzg2N2EyYjZlNWExMTA3ZjUxZmFkNjYzYzQxZDc

In [ ]:
!make runomp

gcc -Ofast -fopenmp -march=native run.c  -lm  -o run
gcc -Ofast -fopenmp -march=native runq.c  -lm  -o runq


In [ ]:
!./run stories15M.bin

Once upon a time, there was a nosy cat named Tom. Tom loved to do fun things. One day, Tom saw a big box of chocolate. He wanted to eat it all by himself.
Tom's friend, a little dog named Max, came over. Max wanted some chocolate too. But Tom did not want to share. They began to fight over the chocolate. They were both very mad.
Then, something unexpected happened. A big bird came and took the chocolate away. Tom and Max stopped fighting. They decided to work together to get the chocolate back. They got the chocolate and shared it. They were happy and became friends again.
achieved tok/s: 719.211823


In [ ]:
# 直接获取已经训练好的pytorch格式的模型，使用pytorch推理
!wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.pt -P out15M


--2023-12-23 08:33:07--  https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.pt
Resolving huggingface.co (huggingface.co)... 13.33.33.55, 13.33.33.110, 13.33.33.20, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/88/4b/884bade32e5ee32eea725c5087af1358179a1bea94a4f6abc3c0470c9610ac38/3da00c0fef684f3f83b457736837c46ab55e92a26662b61d6104de2d271c708d?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27stories15M.pt%3B+filename%3D%22stories15M.pt%22%3B&Expires=1703579587&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMzU3OTU4N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84OC80Yi84ODRiYWRlMzJlNWVlMzJlZWE3MjVjNTA4N2FmMTM1ODE3OWExYmVhOTRhNGY2YWJjM2MwNDcwYzk2MTBhYzM4LzNkYTAwYzBmZWY2ODRmM2Y4M2I0NTc3MzY4MzdjNDZhYjU1ZTkyYTI2NjYyYjYxZDYxMDRkZTJkMjcxYzcwOGQ%7EcmVzcG9uc2UtY29ud

In [ ]:
!python sample.py --checkpoint=out15M/stories15M.pt


Overriding: checkpoint = out15M/stories15M.pt
Once upon a time, there was a little girl named Lily. She loved the color pink and had a pink dress. One day, she was playing outside with her friends when she saw a scary shadow. She looked up and saw a big hot cell on the ceiling. 
Lily's friends didn't see the spider that was trying to escape. They were getting tired of scared and left. Lily was sad because she couldn't see her
---------------


# 自定义tokenizer训练

在上述所有内容中，我们假设自定义 llama 2 标记器具有 32,000 个标记。然而，在许多垂直领域的LLM中，使用这么大的词汇量可能有点大材小用。比如前面的例子，采用llama2训练，需要预处理token;
默认情况下，要对tinystories数据集进行预标记，这里的阶段pretokenize加载 llama 2 分词器（词汇大小 32,000）并使用它将下载的文本转换为整数，并将其保存到文件中。必须按顺序运行：

```
python tinystories.py download
python tinystories.py pretokenize

```

如果您想要一个小型应用程序，那么训练自己的标记器可能会更好。使用较小的词汇，模型具有更少的参数（因为标记embedding表要小得多），推理速度更快（因为要预测的token更少），并且每个示例的平均序列长度也可以得到更小（因为压缩对数据的效率要高得多）。那么看看如何训练自定义分词器。

现在，我们将其更改如下，以训练示例 4096 token分词器：

```
python tinystories.py download
python tinystories.py train_vocab --vocab_size=4096
python tinystories.py pretokenize --vocab_size=4096
```




## 训练词表大小4096tokenizer来预训练模型

In [ ]:
# 训练词表大小为4096的token分词器
# 在TinyStories数据集上训练自定义句子标记器。
# 自定义标记器文件将保存在data/tok{N}目录中，
# 其中N是单词的大小。

# 首先从tinystories文本集中获取前10个数据作为训练token分词器的训练样本数据
# https://github.com/google/sentencepiece
# 然后使用sentencepiece 基于神经网络的文本生成进行无监督文本标记器text tokenizer，采用bpe构建词表
# bpe介绍：https://zhuanlan.zhihu.com/p/424631681
!python tinystories.py train_vocab --vocab_size=4096


Writing temporary file data/tiny.txt with 10 shards...
100% 10/10 [00:13<00:00,  1.38s/it]
Size is: 739.57 MB
Will now train the vocab...
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/tiny.txt
  input_format: text
  model_prefix: data/tok4096
  model_type: BPE
  vocab_size: 4096
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 8
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos

In [ ]:
!ls -lhg data/

total 2.3G
drwxr-xr-x 2 300000513 4.0K May 17  2023 TinyStories_all_data
-rw-r--r-- 1 root      1.5G Dec 23 08:30 TinyStories_all_data.tar.gz
-rw-r--r-- 1 root      740M Dec 23 08:52 tiny.txt
-rw-r--r-- 1 root       63K Dec 23 08:54 tok4096.model
-rw-r--r-- 1 root       53K Dec 23 08:54 tok4096.vocab


In [ ]:
# 对使用该分词器训练的模型，在进行推理时也需要使用对应分词器；且格式必须.bin，导出分词器如下：
!python tokenizer.py --tokenizer-model=data/tok4096.model

In [ ]:
!ls -lhg data/

total 2.3G
drwxr-xr-x 2 300000513 4.0K May 17  2023 TinyStories_all_data
-rw-r--r-- 1 root      1.5G Dec 23 08:30 TinyStories_all_data.tar.gz
-rw-r--r-- 1 root      740M Dec 23 08:52 tiny.txt
drwxr-xr-x 2 root      4.0K Dec 23 09:45 tok4096
-rw-r--r-- 1 root       53K Dec 23 10:10 tok4096.bin
-rw-r--r-- 1 root       63K Dec 23 08:54 tok4096.model
-rw-r--r-- 1 root       53K Dec 23 08:54 tok4096.vocab


In [ ]:
# 对TinyStories数据集通过训练好的分词器进行分词预处理；
# 通过进程池并发异步读取每个分割的文件data*.json进行处理，然后归并，生成pretok.bin文件的位置。
!python tinystories.py pretokenize --vocab_size=4096


```
100% 100000/100000 [01:05<00:00, 1527.63it/s]
100% 100000/100000 [01:04<00:00, 1540.07it/s]
Saved data/tok4096/data17.bin, average seqlen: 196.92
100% 100000/100000 [01:05<00:00, 1517.09it/s]
Saved data/tok4096/data18.bin, average seqlen: 197.16
Saved data/tok4096/data20.bin, average seqlen: 196.84
Saved data/tok4096/data22.bin, average seqlen: 197.14
Saved data/tok4096/data19.bin, average seqlen: 197.12
Saved data/tok4096/data23.bin, average seqlen: 196.60
Saved data/tok4096/data21.bin, average seqlen: 197.03
100% 100000/100000 [01:05<00:00, 1535.79it/s]
100% 100000/100000 [01:04<00:00, 1559.10it/s]
100% 100000/100000 [01:04<00:00, 1546.75it/s]
100% 100000/100000 [01:04<00:00, 1557.84it/s]
100% 100000/100000 [01:04<00:00, 1551.70it/s]
Saved data/tok4096/data24.bin, average seqlen: 196.70
100% 100000/100000 [01:04<00:00, 1558.40it/s]
Saved data/tok4096/data26.bin, average seqlen: 197.27
100% 100000/100000 [01:04<00:00, 1543.41it/s]
100% 100000/100000 [01:04<00:00, 1541.51it/s]
Saved data/tok4096/data25.bin, average seqlen: 196.74
Saved data/tok4096/data27.bin, average seqlen: 196.52
Saved data/tok4096/data28.bin, average seqlen: 197.42
Saved data/tok4096/data31.bin, average seqlen: 197.07
Saved data/tok4096/data30.bin, average seqlen: 197.05
Saved data/tok4096/data29.bin, average seqlen: 197.28
100% 100000/100000 [01:05<00:00, 1533.75it/s]
100% 100000/100000 [01:05<00:00, 1528.44it/s]
100% 100000/100000 [01:05<00:00, 1531.96it/s]
100% 100000/100000 [01:05<00:00, 1524.06it/s]
100% 100000/100000 [01:04<00:00, 1550.01it/s]
100% 100000/100000 [01:05<00:00, 1527.03it/s]
Saved data/tok4096/data32.bin, average seqlen: 196.66
100% 100000/100000 [01:05<00:00, 1536.63it/s]
Saved data/tok4096/data33.bin, average seqlen: 196.58
100% 100000/100000 [01:06<00:00, 1514.14it/s]
Saved data/tok4096/data35.bin, average seqlen: 196.98
Saved data/tok4096/data34.bin, average seqlen: 196.80
Saved data/tok4096/data38.bin, average seqlen: 196.83
Saved data/tok4096/data36.bin, average seqlen: 196.56
Saved data/tok4096/data39.bin, average seqlen: 196.82
Saved data/tok4096/data37.bin, average seqlen: 196.76
100% 100000/100000 [01:04<00:00, 1552.37it/s]
100% 100000/100000 [01:04<00:00, 1562.04it/s]
100% 100000/100000 [01:03<00:00, 1562.85it/s]
100% 100000/100000 [01:04<00:00, 1550.28it/s]
100% 100000/100000 [01:04<00:00, 1552.90it/s]
100% 100000/100000 [01:04<00:00, 1553.28it/s]
Saved data/tok4096/data40.bin, average seqlen: 197.00
Saved data/tok4096/data41.bin, average seqlen: 196.71
Saved data/tok4096/data42.bin, average seqlen: 197.03
100% 100000/100000 [01:05<00:00, 1537.17it/s]
100% 100000/100000 [01:04<00:00, 1542.09it/s]
Saved data/tok4096/data43.bin, average seqlen: 196.85
Saved data/tok4096/data44.bin, average seqlen: 196.72
Saved data/tok4096/data45.bin, average seqlen: 196.97
Saved data/tok4096/data46.bin, average seqlen: 196.88
Saved data/tok4096/data47.bin, average seqlen: 196.91
100% 67871/67871 [00:27<00:00, 2469.11it/s]
Saved data/tok4096/data49.bin, average seqlen: 196.74
100% 100000/100000 [00:41<00:00, 2402.07it/s]
Saved data/tok4096/data48.bin, average seqlen: 196.82
Done.
```


In [ ]:
!ls -lhg data/tok4096

total 1.9G
-rw-r--r-- 1 root 38M Dec 23 09:38 data00.bin
-rw-r--r-- 1 root 38M Dec 23 09:38 data01.bin
-rw-r--r-- 1 root 38M Dec 23 09:38 data02.bin
-rw-r--r-- 1 root 38M Dec 23 09:38 data03.bin
-rw-r--r-- 1 root 38M Dec 23 09:38 data04.bin
-rw-r--r-- 1 root 38M Dec 23 09:38 data05.bin
-rw-r--r-- 1 root 38M Dec 23 09:38 data06.bin
-rw-r--r-- 1 root 38M Dec 23 09:38 data07.bin
-rw-r--r-- 1 root 38M Dec 23 09:39 data08.bin
-rw-r--r-- 1 root 38M Dec 23 09:39 data09.bin
-rw-r--r-- 1 root 38M Dec 23 09:39 data10.bin
-rw-r--r-- 1 root 38M Dec 23 09:39 data11.bin
-rw-r--r-- 1 root 38M Dec 23 09:39 data12.bin
-rw-r--r-- 1 root 38M Dec 23 09:39 data13.bin
-rw-r--r-- 1 root 38M Dec 23 09:39 data14.bin
-rw-r--r-- 1 root 38M Dec 23 09:39 data15.bin
-rw-r--r-- 1 root 38M Dec 23 09:40 data16.bin
-rw-r--r-- 1 root 38M Dec 23 09:40 data17.bin
-rw-r--r-- 1 root 38M Dec 23 09:40 data18.bin
-rw-r--r-- 1 root 38M Dec 23 09:40 data19.bin
-rw-r--r-- 1 root 38M Dec 23 09:40 data20.bin
-rw-r--r-- 1 root 38M D

值得注意的是，专门在tinystories 上训练的 4096 个词汇大小创建的整数序列，每个示例的序列长度与默认的 Llama 2 标记器的 32000 个标记的序列长度大致相同！这意味着定制的分词器能够更好地适应我们的特定文本，并且可以非常有效地压缩它。所以训练的模型更小、更快。



现在已经使用自定义标记器对数据集进行了分词tokenizer预处理，我们可以训练模型了。训练脚本train.py不关心确切的token，它只关心词汇量大小，以便它可以正确初始化模型。因此，在训练模型时，请确保传入



In [ ]:
#!python -m train.py --compile=False --eval_iters=1 --batch_size=90 --dtype=float16
!python -m train.py --vocab_source=custom --vocab_size=4096 --dtype=float16 --eval_iters=10 --batch_size=128

# 从最后训练保持的ckpt开始恢复训练
#!python -m train.py --vocab_source=custom --vocab_size=4096 --dtype=float16 --eval_iters=10 --batch_size=128 --init_from resume

# 根据训练模型的参数 以及结合硬件 预估训练时间：
# https://zhuanlan.zhihu.com/p/624740065

流式输出内容被截断，只能显示最后 5000 行内容。
47942 | loss 1.1854 | lr 2.702711e-04 | 700.95ms | mfu 2.88%
47943 | loss 1.2370 | lr 2.702632e-04 | 707.31ms | mfu 2.88%
47944 | loss 1.1772 | lr 2.702553e-04 | 697.65ms | mfu 2.88%
47945 | loss 1.1780 | lr 2.702474e-04 | 703.40ms | mfu 2.88%
47946 | loss 1.1943 | lr 2.702395e-04 | 708.27ms | mfu 2.88%
47947 | loss 1.2036 | lr 2.702316e-04 | 696.58ms | mfu 2.88%
47948 | loss 1.2254 | lr 2.702237e-04 | 704.44ms | mfu 2.88%
47949 | loss 1.2005 | lr 2.702158e-04 | 704.98ms | mfu 2.88%
47950 | loss 1.2153 | lr 2.702079e-04 | 699.82ms | mfu 2.88%
47951 | loss 1.2022 | lr 2.702000e-04 | 699.16ms | mfu 2.88%
47952 | loss 1.1752 | lr 2.701921e-04 | 697.61ms | mfu 2.89%
47953 | loss 1.1902 | lr 2.701842e-04 | 699.37ms | mfu 2.89%
47954 | loss 1.2295 | lr 2.701763e-04 | 698.07ms | mfu 2.89%
47955 | loss 1.1385 | lr 2.701683e-04 | 701.39ms | mfu 2.89%
47956 | loss 1.2217 | lr 2.701604e-04 | 698.24ms | mfu 2.89%
47957 | loss 1.2136 | lr 2.701525e-04 | 701.35ms | mfu 2.8

In [ ]:
!ls -lgh out/

In [ ]:
# 使用训练好的toknizer 和 模型 文件进行推理
!./run out/model.bin -z data/tok4096.bin


## 训练词表大小512tokenizer来预训练模型

In [ ]:
# 训练词表大小为512的token分词器
# 在TinyStories数据集上训练自定义句子标记器。
# 自定义标记器文件将保存在data/tok{N}目录中，
# 其中N是单词的大小。

# 首先从tinystories文本集中获取前10个数据作为训练token分词器的训练样本数据
# https://github.com/google/sentencepiece
# 然后使用sentencepiece 基于神经网络的文本生成进行无监督文本标记器text tokenizer，采用bpe构建词表
# bpe介绍：https://zhuanlan.zhihu.com/p/424631681
!python tinystories.py train_vocab --vocab_size=512


Writing temporary file data/tiny.txt with 10 shards...
100% 10/10 [00:14<00:00,  1.42s/it]
Size is: 739.57 MB
Will now train the vocab...
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/tiny.txt
  input_format: text
  model_prefix: data/tok512
  model_type: BPE
  vocab_size: 512
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 8
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 1
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 1
  required_chars: 
  byte_fallback: 1
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_p

In [ ]:
!ls -lhg data/

total 1.5G
drwxr-xr-x 2 300000513 4.0K May 17  2023 TinyStories_all_data
-rw-r--r-- 1 root      1.5G Dec 24 14:48 TinyStories_all_data.tar.gz
-rw-r--r-- 1 root      7.4K Dec 24 14:53 tok512.model
-rw-r--r-- 1 root      4.4K Dec 24 14:53 tok512.vocab


In [ ]:
# 对TinyStories数据集通过训练好的分词器进行分词预处理；
# 通过进程池并发异步读取每个分割的文件data*.json进行处理，然后归并，生成pretok.bin文件的位置。
!python tinystories.py pretokenize --vocab_size=512


```
100% 100000/100000 [01:27<00:00, 1145.19it/s]
100% 100000/100000 [01:27<00:00, 1148.00it/s]
100% 100000/100000 [01:26<00:00, 1159.62it/s]
100% 100000/100000 [01:26<00:00, 1154.90it/s]
100% 100000/100000 [01:27<00:00, 1148.17it/s]
Saved data/tok512/data16.bin, average seqlen: 371.83
Saved data/tok512/data20.bin, average seqlen: 372.41
Saved data/tok512/data18.bin, average seqlen: 373.06
Saved data/tok512/data19.bin, average seqlen: 372.90
Saved data/tok512/data17.bin, average seqlen: 372.45
Saved data/tok512/data22.bin, average seqlen: 373.08
Saved data/tok512/data21.bin, average seqlen: 372.76
Saved data/tok512/data23.bin, average seqlen: 372.03
100% 100000/100000 [01:23<00:00, 1194.04it/s]
100% 100000/100000 [01:24<00:00, 1179.93it/s]
100% 100000/100000 [01:24<00:00, 1184.17it/s]
100% 100000/100000 [01:24<00:00, 1178.59it/s]
100% 100000/100000 [01:25<00:00, 1175.94it/s]
100% 100000/100000 [01:25<00:00, 1172.09it/s]
100% 100000/100000 [01:24<00:00, 1183.26it/s]
Saved data/tok512/data25.bin, average seqlen: 372.28
Saved data/tok512/data24.bin, average seqlen: 372.24
100% 100000/100000 [01:25<00:00, 1172.69it/s]
Saved data/tok512/data26.bin, average seqlen: 373.17
Saved data/tok512/data27.bin, average seqlen: 371.96
Saved data/tok512/data28.bin, average seqlen: 373.53
Saved data/tok512/data29.bin, average seqlen: 373.35
Saved data/tok512/data30.bin, average seqlen: 372.94
Saved data/tok512/data31.bin, average seqlen: 372.81
100% 100000/100000 [01:25<00:00, 1175.18it/s]
100% 100000/100000 [01:25<00:00, 1176.18it/s]
100% 100000/100000 [01:24<00:00, 1183.63it/s]
100% 100000/100000 [01:24<00:00, 1182.32it/s]
100% 100000/100000 [01:24<00:00, 1188.17it/s]
100% 100000/100000 [01:24<00:00, 1180.56it/s]
100% 100000/100000 [01:24<00:00, 1185.62it/s]
Saved data/tok512/data33.bin, average seqlen: 372.00
Saved data/tok512/data32.bin, average seqlen: 372.12
100% 100000/100000 [01:25<00:00, 1169.34it/s]
Saved data/tok512/data34.bin, average seqlen: 372.26
Saved data/tok512/data35.bin, average seqlen: 372.80
Saved data/tok512/data37.bin, average seqlen: 372.13
Saved data/tok512/data36.bin, average seqlen: 371.80
Saved data/tok512/data38.bin, average seqlen: 372.54
Saved data/tok512/data39.bin, average seqlen: 372.33
100% 100000/100000 [01:24<00:00, 1181.72it/s]
100% 100000/100000 [01:25<00:00, 1173.78it/s]
100% 100000/100000 [01:24<00:00, 1181.08it/s]
100% 100000/100000 [01:25<00:00, 1174.15it/s]
100% 100000/100000 [01:24<00:00, 1181.09it/s]
100% 100000/100000 [01:25<00:00, 1169.86it/s]
100% 100000/100000 [01:25<00:00, 1171.82it/s]
Saved data/tok512/data40.bin, average seqlen: 372.73
Saved data/tok512/data41.bin, average seqlen: 372.10
100% 100000/100000 [01:25<00:00, 1166.42it/s]
Saved data/tok512/data42.bin, average seqlen: 372.71
Saved data/tok512/data43.bin, average seqlen: 372.49
Saved data/tok512/data44.bin, average seqlen: 372.15
Saved data/tok512/data46.bin, average seqlen: 372.51
Saved data/tok512/data45.bin, average seqlen: 372.60
Saved data/tok512/data47.bin, average seqlen: 372.57
100% 67871/67871 [00:34<00:00, 1961.79it/s]
Saved data/tok512/data49.bin, average seqlen: 372.17
100% 100000/100000 [00:52<00:00, 1888.93it/s]
Saved data/tok512/data48.bin, average seqlen: 372.39
Done.
```

In [ ]:
!ls -lhg data/tok512

total 3.5G
-rw-r--r-- 1 root 72M Dec 24 14:56 data00.bin
-rw-r--r-- 1 root 72M Dec 24 14:56 data01.bin
-rw-r--r-- 1 root 72M Dec 24 14:56 data02.bin
-rw-r--r-- 1 root 72M Dec 24 14:56 data03.bin
-rw-r--r-- 1 root 72M Dec 24 14:56 data04.bin
-rw-r--r-- 1 root 72M Dec 24 14:56 data05.bin
-rw-r--r-- 1 root 72M Dec 24 14:56 data06.bin
-rw-r--r-- 1 root 72M Dec 24 14:56 data07.bin
-rw-r--r-- 1 root 72M Dec 24 14:57 data08.bin
-rw-r--r-- 1 root 72M Dec 24 14:57 data09.bin
-rw-r--r-- 1 root 71M Dec 24 14:57 data10.bin
-rw-r--r-- 1 root 71M Dec 24 14:57 data11.bin
-rw-r--r-- 1 root 72M Dec 24 14:57 data12.bin
-rw-r--r-- 1 root 72M Dec 24 14:57 data13.bin
-rw-r--r-- 1 root 71M Dec 24 14:57 data14.bin
-rw-r--r-- 1 root 71M Dec 24 14:57 data15.bin
-rw-r--r-- 1 root 71M Dec 24 14:59 data16.bin
-rw-r--r-- 1 root 72M Dec 24 14:59 data17.bin
-rw-r--r-- 1 root 72M Dec 24 14:59 data18.bin
-rw-r--r-- 1 root 72M Dec 24 14:59 data19.bin
-rw-r--r-- 1 root 72M Dec 24 14:59 data20.bin
-rw-r--r-- 1 root 72M D

In [ ]:
!python -m train.py \
    --out_dir="outmini" \
    --batch_size=128 \
    --max_seq_len=512 \
    --gradient_accumulation_steps=1 \
    --vocab_source="custom" \
    --vocab_size=512 \
    --dim=64 \
    --n_layers=5 \
    --n_heads=8 \
    --n_kv_heads=4 \
    --multiple_of=4 \
    --learning_rate=1e-3 \
    --dropout=0.05 \
    --weight_decay=0.01 \
    --max_iters=100000 \
    --beta2=0.99 \
    --warmup_iters=1000 \
    --eval_interval=2000 \
    --eval_iters=100 \
    --dtype=float16 \
    --compile=True


流式输出内容被截断，只能显示最后 5000 行内容。
95017 | loss 1.4118 | lr 6.238009e-06 | 85.67ms | mfu 0.88%
95018 | loss 1.4270 | lr 6.235511e-06 | 84.42ms | mfu 0.88%
95019 | loss 1.4479 | lr 6.233013e-06 | 84.48ms | mfu 0.88%
95020 | loss 1.4542 | lr 6.230516e-06 | 84.96ms | mfu 0.88%
95021 | loss 1.4024 | lr 6.228019e-06 | 83.02ms | mfu 0.88%
95022 | loss 1.4203 | lr 6.225523e-06 | 84.28ms | mfu 0.88%
95023 | loss 1.4027 | lr 6.223027e-06 | 85.19ms | mfu 0.88%
95024 | loss 1.4397 | lr 6.220532e-06 | 84.23ms | mfu 0.88%
95025 | loss 1.4333 | lr 6.218037e-06 | 84.44ms | mfu 0.88%
95026 | loss 1.4154 | lr 6.215543e-06 | 84.10ms | mfu 0.88%
95027 | loss 1.4224 | lr 6.213049e-06 | 85.10ms | mfu 0.88%
95028 | loss 1.4304 | lr 6.210556e-06 | 84.56ms | mfu 0.88%
95029 | loss 1.4411 | lr 6.208063e-06 | 85.89ms | mfu 0.88%
95030 | loss 1.4527 | lr 6.205571e-06 | 84.30ms | mfu 0.88%
95031 | loss 1.4384 | lr 6.203079e-06 | 84.39ms | mfu 0.88%
95032 | loss 1.4620 | lr 6.200588e-06 | 84.63ms | mfu 0.88%
95033 | loss 

# llama2-zh.c


In [ ]:
%cd /content
!git clone https://github.com/chenyangMl/llama2.c-zh.git

/content
Cloning into 'llama2.c-zh'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 122 (delta 57), reused 94 (delta 31), pack-reused 0
Receiving objects: 100% (122/122), 1.96 MiB | 6.85 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/llama2.c-zh


In [ ]:
%cd llama2.c-zh

/content/llama2.c-zh


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 下载中英文数据
!python dataProcess/tinystories.py download


data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [01:19<00:00, 20.2MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [ ]:
# tokenizer 数据
!python dataProcess/tinystories.py pretokenize

In [ ]:
!ls -ghl data/TinyStories_all_data-en/

total 9.6G
-rw-r--r-- 1 root       65M Dec 28 12:49 data00.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data00.json
-rw-r--r-- 1 root       64M Dec 28 12:49 data01.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data01.json
-rw-r--r-- 1 root       65M Dec 28 12:50 data02.bin
-rw-r--r-- 1 300000513 135M May 17  2023 data02.json
-rw-r--r-- 1 root       65M Dec 28 12:49 data03.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data03.json
-rw-r--r-- 1 root       65M Dec 28 12:49 data04.bin
-rw-r--r-- 1 300000513 135M May 17  2023 data04.json
-rw-r--r-- 1 root       65M Dec 28 12:49 data05.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data05.json
-rw-r--r-- 1 root       65M Dec 28 12:50 data06.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data06.json
-rw-r--r-- 1 root       65M Dec 28 12:50 data07.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data07.json
-rw-r--r-- 1 root       65M Dec 28 12:49 data08.bin
-rw-r--r-- 1 300000513 134M May 17  2023 data08.json
-rw-r--r-- 1 root       64M Dec 28 12:49 dat

In [ ]:
!ls -ghl data/TinyStories_all_data-zh/

total 1.2G
-rw-r--r-- 1 root  38M Dec 28 13:31 data00.bin
-rw-r--r-- 1 staff 71M Aug 12 15:31 data00.json
-rw-r--r-- 1 root  37M Dec 28 13:31 data01.bin
-rw-r--r-- 1 staff 70M Aug 13 01:18 data01.json
-rw-r--r-- 1 root  37M Dec 28 13:31 data02.bin
-rw-r--r-- 1 staff 70M Aug 12 15:32 data02.json
-rw-r--r-- 1 root  38M Dec 28 13:31 data03.bin
-rw-r--r-- 1 staff 71M Aug 12 15:33 data03.json
-rw-r--r-- 1 root  37M Dec 28 13:31 data04.bin
-rw-r--r-- 1 staff 71M Aug 12 15:34 data04.json
-rw-r--r-- 1 root  37M Dec 28 13:31 data05.bin
-rw-r--r-- 1 staff 70M Aug 12 15:35 data05.json
-rw-r--r-- 1 root  39M Dec 28 13:31 data06.bin
-rw-r--r-- 1 staff 73M Aug 12 15:38 data06.json
-rw-r--r-- 1 root  37M Dec 28 13:31 data07.bin
-rw-r--r-- 1 staff 70M Aug 12 15:39 data07.json
-rw-r--r-- 1 root  37M Dec 28 13:31 data08.bin
-rw-r--r-- 1 staff 70M Aug 12 15:40 data08.json
-rw-r--r-- 1 root  37M Dec 28 13:31 data09.bin
-rw-r--r-- 1 staff 70M Aug 12 15:41 data09.json
-rw-r--r-- 1 root  22M Dec 28 13:29 dat

In [ ]:
#!python -m train.py --eval_iters=10 --batch_size=128 --dtype=float16
#!python -m train.py --eval_iters=10 --batch_size=10 --dtype=float16
!python -m train.py --eval_iters=1 --batch_size=70 --dtype=float16


Streaming output truncated to the last 5000 lines.
58634 | loss 0.2364 | lr 3.370409e-04 | 194.26ms | mfu 2.29%
58635 | loss 0.2060 | lr 3.370360e-04 | 197.67ms | mfu 2.28%
58636 | loss 0.2120 | lr 3.370310e-04 | 195.70ms | mfu 2.28%
58637 | loss 0.2458 | lr 3.370261e-04 | 195.12ms | mfu 2.29%
58638 | loss 0.1925 | lr 3.370211e-04 | 197.19ms | mfu 2.29%
58639 | loss 0.2190 | lr 3.370162e-04 | 199.10ms | mfu 2.28%
58640 | loss 0.2259 | lr 3.370113e-04 | 195.84ms | mfu 2.28%
58641 | loss 0.2200 | lr 3.370063e-04 | 194.16ms | mfu 2.29%
58642 | loss 0.2404 | lr 3.370014e-04 | 199.28ms | mfu 2.28%
58643 | loss 0.2384 | lr 3.369964e-04 | 196.67ms | mfu 2.28%
58644 | loss 0.2218 | lr 3.369915e-04 | 193.59ms | mfu 2.29%
58645 | loss 0.2561 | lr 3.369865e-04 | 197.99ms | mfu 2.28%
58646 | loss 0.2756 | lr 3.369816e-04 | 197.92ms | mfu 2.28%
58647 | loss 0.2125 | lr 3.369767e-04 | 196.67ms | mfu 2.28%
58648 | loss 0.2510 | lr 3.369717e-04 | 195.06ms | mfu 2.28%
58649 | loss 0.2223 | lr 3.369668e

## 小说数据集训练
- https://github.com/shjwudp/shu/tree/master/books


In [ ]:
!python dataProcess/tinyfictions.py download

In [ ]:
!python dataProcess/tinyfictions.py pretokenize


In [ ]:
# 训练. 修改batch_size=256, vocab_size=自定义的词表长度, 其他根据需要进行调整
#!python train.py
# 单卡
#!python -m train.py --eval_iters=10 --batch_size=10 --dtype=float16
!python -m train.py --eval_iters=1 --batch_size=70 --dtype=float16


# baby-llama2-chinese

In [ ]:
%cd /content
!git clone https://github.com/DLLXW/baby-llama2-chinese.git

/content
Cloning into 'baby-llama2-chinese'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 73 (delta 11), reused 11 (delta 11), pack-reused 47
Receiving objects: 100% (73/73), 479.85 KiB | 19.19 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
%cd /content/baby-llama2-chinese

/content/baby-llama2-chinese


# 更多中文数据集和模型参考

- https://github.com/crownpku/awesome-chinese-nlp#corpus-%E4%B8%AD%E6%96%87%E8%AF%AD%E6%96%99
- https://github.com/brightmart/nlp_chinese_corpus
- NLP民工的乐园(百科全书): **https://github.com/fighting41love/funNLP**
- 大模型汇总： https://github.com/chenking2020/FindTheChatGPTer
- 超赞的超大规模中文语料集： https://github.com/esbatmop/MNBVC